In [1]:
import sys
sys.executable

'/home/isaac/miniconda3/envs/3dvision/bin/python'

In [3]:
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
import socket
import glob

import os
import sys
BASE_DIR = os.path.dirname(os.getcwd())
BASE_DIR = os.path.join(BASE_DIR, "pointnet_semseg_vkitti" )
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'utils'))
sys.path.append(os.path.join(BASE_DIR, 'data'))
import provider
import tf_util
from model import *


In [26]:
BATCH_SIZE = 24
BATCH_SIZE_EVAL = 24
NUM_POINT = 4096
MAX_EPOCH = 50
BASE_LEARNING_RATE = 0.001
GPU_INDEX = 0
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 300000
DECAY_RATE = 0.5

LOG_DIR = 'log'
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp model.py %s' % (LOG_DIR)) # bkp of model def
os.system('cp train.py %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
# LOG_FOUT.write(str(FLAGS)+'\n')

MAX_NUM_POINT = 4096
NUM_CLASSES = 13

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
#BN_DECAY_DECAY_STEP = float(DECAY_STEP * 2)
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()

In [5]:
folders = glob.glob('./data/vkitti3d_dataset_v1.0/*')

In [6]:
ALL_FILES = []
for folder in folders:
    np_arrays = glob.glob(folder + "/*")
    ALL_FILES += np_arrays

In [7]:
ALL_FILES

['./data/vkitti3d_dataset_v1.0/01/0001_00000.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00012.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00024.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00036.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00048.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00060.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00072.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00085.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00097.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00109.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00121.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00133.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00145.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00157.npy',
 './data/vkitti3d_dataset_v1.0/01/0001_00170.npy',
 './data/vkitti3d_dataset_v1.0/02/0001_00230.npy',
 './data/vkitti3d_dataset_v1.0/02/0001_00243.npy',
 './data/vkitti3d_dataset_v1.0/02/0001_00257.npy',
 './data/vkitti3d_dataset_v1.0/02/0001_00270.npy',
 './data/vkitti3d_dataset_v1.0/

In [9]:
len(ALL_FILES)

90

In [10]:
# Load ALL data
total_data = np.empty((90, 4096, 6))
total_label = np.empty((90, 4096))

for i, file in enumerate(ALL_FILES):
    f = np.load(file)  # shape: (401326, 7)
    data = f[:,:6]
    data = np.reshape(data, (1, data.shape[0], data.shape[1]))
    label = f[:,-1]
    label = np.reshape(label, (1, label.shape[0]))
    
    idxs = np.arange(0, data.shape[1])
    np.random.shuffle(idxs)
    total_data[i,:,:] = data[:, idxs[:4096], :]
    total_label[i,:] = label[:, idxs[:4096]]
    
print(total_data.shape)
print(total_label.shape)

(90, 4096, 6)
(90, 4096)


In [11]:
features = ["x","y","z","r","g","b"]
for i in range(6): 
    print(features[i] + "_range :", np.min(total_data[:, :, i]), np.max(total_data[:, :, i]))

x_range : 2.1171770095825195 149.8768768310547
y_range : -96.41452026367188 95.5736312866211
z_range : -3.9555277824401855 30.010095596313477
r_range : 0.0 255.0
g_range : 0.0 255.0
b_range : 0.0 255.0


In [12]:
X = total_data
y = total_label

X.shape, y.shape

((90, 4096, 6), (90, 4096))

In [13]:
xmin = []
xmax = []
for i in range(6):
    xmin.append(np.min(X[:,:,i]))
    xmax.append(np.max(X[:,:,i]))
    
print(xmin)
print(xmax)

[2.1171770095825195, -96.41452026367188, -3.9555277824401855, 0.0, 0.0, 0.0]
[149.8768768310547, 95.5736312866211, 30.010095596313477, 255.0, 255.0, 255.0]


In [14]:
X_normal = np.zeros(X.shape)
for i in range(6):
    X_normal[:,:,i] = (X[:,:,i] - xmin[i]) / (xmax[i] - xmin[i])

In [15]:
features = ["x","y","z","r","g","b"]
for i in range(6): 
    print(features[i] + "_range :", np.min(X_normal[:, :, i]), np.max(X_normal[:, :, i]))

x_range : 0.0 1.0
y_range : 0.0 1.0
z_range : 0.0 1.0
r_range : 0.0 1.0
g_range : 0.0 1.0
b_range : 0.0 1.0


In [29]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(X_normal, y, test_size=0.26, random_state=42)

print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

(66, 4096, 6) (66, 4096)
(24, 4096, 6) (24, 4096)


In [31]:

def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)


def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

def train():
    with tf.Graph().as_default():
        with tf.device('/gpu:'+str(GPU_INDEX)):
            pointclouds_pl, labels_pl = placeholder_inputs(BATCH_SIZE, NUM_POINT)
            is_training_pl = tf.placeholder(tf.bool, shape=())
            
            # Note the global_step=batch parameter to minimize. 
            # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
            batch = tf.Variable(0)
            bn_decay = get_bn_decay(batch)
            tf.summary.scalar('bn_decay', bn_decay)

            # Get model and loss 
            pred = get_model(pointclouds_pl, is_training_pl, bn_decay=bn_decay)
            loss = get_loss(pred, labels_pl)
            tf.summary.scalar('loss', loss)

            correct = tf.equal(tf.argmax(pred, 2), tf.to_int64(labels_pl))
            accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE*NUM_POINT)
            tf.summary.scalar('accuracy', accuracy)

            # Get training operator
            learning_rate = get_learning_rate(batch)
            tf.summary.scalar('learning_rate', learning_rate)
            if OPTIMIZER == 'momentum':
                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
            elif OPTIMIZER == 'adam':
                optimizer = tf.train.AdamOptimizer(learning_rate)
            train_op = optimizer.minimize(loss, global_step=batch)
            
            # Add ops to save and restore all the variables.
            saver = tf.train.Saver()
            
        # Create a session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        config.log_device_placement = True
        sess = tf.Session(config=config)

        # Add summary writers
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                                  sess.graph)
        test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))

        # Init variables
        init = tf.global_variables_initializer()
        sess.run(init, {is_training_pl:True})

        ops = {'pointclouds_pl': pointclouds_pl,
               'labels_pl': labels_pl,
               'is_training_pl': is_training_pl,
               'pred': pred,
               'loss': loss,
               'train_op': train_op,
               'merged': merged,
               'step': batch}

        for epoch in range(MAX_EPOCH):
            log_string('**** EPOCH %03d ****' % (epoch))
            sys.stdout.flush()
             
            train_one_epoch(sess, ops, train_writer)
            eval_one_epoch(sess, ops, test_writer)
            
            # Save the variables to disk.
            if epoch % 10 == 0:
                save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
                log_string("Model saved in file: %s" % save_path)



def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    
    log_string('----')
    current_data, current_label, _ = provider.shuffle_data(train_data[:,0:NUM_POINT,:], train_label) 
    
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    
    for batch_idx in range(num_batches):
        if batch_idx % 100 == 0:
            print('Current batch/total batch num: %d/%d'%(batch_idx,num_batches))
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        
        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training,}
        summary, step, _, loss_val, pred_val = sess.run([ops['merged'], ops['step'], ops['train_op'], ops['loss'], ops['pred']],
                                         feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += loss_val
    
    log_string('mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('accuracy: %f' % (total_correct / float(total_seen)))

        
def eval_one_epoch(sess, ops, test_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    log_string('----')
    current_data = test_data[:,0:NUM_POINT,:]
    current_label = np.squeeze(test_label)
    
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE_EVAL
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE_EVAL
        end_idx = (batch_idx+1) * BATCH_SIZE_EVAL

        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training}
        summary, step, loss_val, pred_val = sess.run([ops['merged'], ops['step'], ops['loss'], ops['pred']],
                                      feed_dict=feed_dict)
        test_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += (BATCH_SIZE_EVAL*NUM_POINT)
        loss_sum += (loss_val*BATCH_SIZE_EVAL)
        for i in range(start_idx, end_idx):
            for j in range(NUM_POINT):
                l = int(current_label[i, j])
                total_seen_class[l] += 1
                total_correct_class[l] += (pred_val[i-start_idx, j] == l)
            
    log_string('eval mean loss: %f' % (loss_sum / float(total_seen/NUM_POINT)))
    log_string('eval accuracy: %f'% (total_correct / float(total_seen)))
    log_string('eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))
         

if __name__ == "__main__":
    train()
    LOG_FOUT.close()

Tensor("fc2/Relu:0", shape=(24, 128), dtype=float32, device=/device:GPU:0)
**** EPOCH 000 ****
----
Current batch/total batch num: 0/2
mean loss: 2.537423
accuracy: 0.254990
----
eval mean loss: 43.578381
eval accuracy: 0.268188
eval avg class acc: 0.150831
Model saved in file: log/model.ckpt
**** EPOCH 001 ****
----
Current batch/total batch num: 0/2
mean loss: 1.166659
accuracy: 0.697266
----
eval mean loss: 4.697268
eval accuracy: 0.423299
eval avg class acc: 0.293540
**** EPOCH 002 ****
----
Current batch/total batch num: 0/2
mean loss: 0.867883
accuracy: 0.757619
----
eval mean loss: 1.565815
eval accuracy: 0.690664
eval avg class acc: 0.366182
**** EPOCH 003 ****
----
Current batch/total batch num: 0/2
mean loss: 0.702951
accuracy: 0.799886
----
eval mean loss: 1.117834
eval accuracy: 0.748820
eval avg class acc: 0.385326
**** EPOCH 004 ****
----
Current batch/total batch num: 0/2
mean loss: 0.613740
accuracy: 0.826279
----
eval mean loss: 0.902869
eval accuracy: 0.788208
eval av